In [50]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re

In [51]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

In [52]:
#test

tokens = tokenizer.encode('This was very much ok, not above nor below standards', return_tensors='pt')

In [53]:
result = model(tokens)
print(result)

SequenceClassifierOutput(loss=None, logits=tensor([[-0.8097,  1.1630,  2.1242, -0.0501, -2.2006]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)


In [54]:
# 1-5, 1 being most negative, 5 being most positive
sentiment = int(torch.argmax(result.logits)) + 1
sentiment


3

In [55]:
r = requests.get('https://www.yelp.com/biz/whiskey-kitchen-raleigh')
soup = BeautifulSoup(r.text, 'html.parser')
regex = re.compile('.*comment.*')
results = soup.find_all('p', {'class':regex})
reviews = [result.text for result in results]



In [56]:
results[0].text

"Food is delicious and I've never had fries stay crispy during my whole meal! Service was excellent and fast"

In [57]:
import numpy as np 
import pandas as pd 


In [58]:
df = pd.DataFrame(np.array(reviews), columns=['review'])

In [59]:
df['review'].iloc[0]

"Food is delicious and I've never had fries stay crispy during my whole meal! Service was excellent and fast"

In [68]:
df_1 = pd.read_csv('Users/Chetan/Desktop/Projects/Sentiment Analysis/test.csv')
df_1 = df_1.head(100)
df_1.head()

,text,label
0,I always wrote this series off as being a comp...,0
1,1st watched 12/7/2002 - 3 out of 10(Dir-Steve ...,0
2,This movie was so poorly written and directed ...,0
3,The most interesting thing about Miryang (Secr...,1
4,"when i first read about ""berlin am meer"" i did...",0


In [69]:
reviews = df['review'].tolist()

In [70]:
def sentiment_track(review):
    tokens = tokenizer.encode(review, return_tensors='pt', truncation=True, padding=True, max_length=512)
    result = model(tokens)
    return int(torch.argmax(result.logits))

In [71]:
df_1['sentiment'] = df_1['text'].apply(lambda x: sentiment_track(x))

In [72]:
print(df_1.head())


                                                text  label  sentiment
0  I always wrote this series off as being a comp...      0          4
1  1st watched 12/7/2002 - 3 out of 10(Dir-Steve ...      0          2
2  This movie was so poorly written and directed ...      0          1
3  The most interesting thing about Miryang (Secr...      1          3
4  when i first read about "berlin am meer" i did...      0          1
